In [0]:
%pip install prophet==1.1.5 \
    convertdate==2.4.0 \
    holidays==0.34 \
    cmdstanpy==1.2.3


In [0]:
# ==============================================================================
# 1. IMPORT LIBRATIES
# ==============================================================================

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import mlflow
import pickle
from prophet import Prophet
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import lit

In [0]:
# ==============================================================================
# 2. CONFIGURATION AND UTILS
# ==============================================================================

INPUT_TABLE_NAME = "retail_feature_engg_done"
FORECAST_DAYS = 30
FORECAST_TYPE = 'daily_revenue' # options: 'daily_revenue', 'daily_orders'
TARGET_METRIC = 'mape' # Metric to log/optimize

# DATABASE CONNECTION
pg_host = dbutils.secrets.get(scope="postgres-secrets", key="host")
pg_port = dbutils.secrets.get(scope="postgres-secrets", key="port")
pg_database = dbutils.secrets.get(scope="postgres-secrets", key="database")
pg_username = dbutils.secrets.get(scope="postgres-secrets", key="username")
pg_password = dbutils.secrets.get(scope="postgres-secrets", key="password")

jdbc_url = f"jdbc:postgresql://{pg_host}:{pg_port}/{pg_database}"
CONNECTION_PROPERTIES = {
    "user": pg_username,
    "password": pg_password,
    "driver": "org.postgresql.Driver"
}

In [0]:
# ==============================================================================
# 3. READ AND CONVERT TO PANDAS
# ==============================================================================

def read_and_prepare_data(table_name: str, forecast_type: str, spark: SparkSession) -> pd.DataFrame:
    """
    Reads data from PostgreSQL using PySpark, converts to Pandas, and prepares 
    the DataFrame for Prophet (aggregation, cleaning, feature engineering).
    """
    print(f"Reading data from PostgreSQL table: {table_name}...")
    
    # 1. READ DATA via PySpark
    spark_df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=CONNECTION_PROPERTIES)
    df = spark_df.toPandas()
    
    
    # Convert purchase_datetime to datetime
    df['purchase_datetime'] = pd.to_datetime(df['purchase_datetime'])

    print(f"Loaded {len(df)} records...")
    
    # Filter out sparse early data
    start_date = pd.to_datetime('2017-09-01')
    df = df[df['purchase_datetime'] >= start_date].copy()
    
    print(f"✓ Filtered to start from: {start_date}")

    # 2. Aggregate based on forecast type
    if forecast_type == 'daily_revenue':
        df = df.groupby(df['purchase_datetime'].dt.date).agg({'total_payment_value':'sum'}).reset_index()
        df.columns = ['ds', 'y']
        print(f"Aggregated to daily revenue: {len(df)} days")

    elif forecast_type == 'daily_orders':
        df = df.groupby(df['purchase_datetime'].dt.date).agg({'order_id':'nunique'}).reset_index()
        df.columns = ['ds', 'y']
        print(f"Aggregated to daily orders: {len(df)} days")

    else:
        raise ValueError(f"Unknown forecast_type: {forecast_type}")

    # Ensure column 'ds' is datetime
    df['ds'] = pd.to_datetime(df['ds'])

    # 3. Clean and Transform
    # Remove extreme outliers using IQR
    q1, q3 = df['y'].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    df['y'] = df['y'].clip(lower, upper)
    
    # Log-transform target to stabilize variance
    df['y_raw'] = df['y'] # Keeping raw value for inverse transformation
    df['y'] = np.log1p(df['y'])
    
    # 4. Add Regressors
    df['weekday'] = df['ds'].dt.weekday
    df['month'] = df['ds'].dt.month
    df['is_weekend'] = df['ds'].dt.weekday >= 5
    
    print(f"✓ Target variable range: {df['y'].min():.2f} to {df['y'].max():.2f}")
    
    return df


In [0]:
# ==============================================================================
# 4. BUILD AND FIT PROPHET MODEL
# ==============================================================================

def train_prophet_model(df):
    """
    Create and fit Prophet model and return the model object.
    """
    model = Prophet(
        seasonality_mode = 'multiplicative',
        changepoint_prior_scale = 0.05,
        seasonality_prior_scale = 10,
        yearly_seasonality = True,
        weekly_seasonality = True,
        daily_seasonality = False)
    
    # Add Brazilian holidays
    model.add_country_holidays(country_name='BR')
    print("Added Brazilian Holidays")

    # Add regressors
    model.add_regressor('weekday')
    model.add_regressor('month')
    model.add_regressor('is_weekend')

    # Fit Model
    print("Training Prophet Model")
    model.fit(df)
    print("Model trained successfully")

    return model

In [0]:
# ==============================================================================
# 5. GENERATE AND INVERSE-TRANSFORM FORECASTS
# ==============================================================================

def generate_and_postprocess_forecast(model, df: pd.DataFrame, periods: int) -> pd.DataFrame:
    """
    Generate future predictions and apply inverse log-transform.
    """
    # 1. Create future dataframe and add regressors
    future = model.make_future_dataframe(periods=periods, include_history=False) # Only future periods
    
    # Re-apply regressors to the future dataframe
    future['weekday'] = future['ds'].dt.weekday
    future['month'] = future['ds'].dt.month
    future['is_weekend'] = future['ds'].dt.weekday >= 5
    
    # 2. Predict
    forecast_future = model.predict(future)

    # 3. Generate predictions for the historical period (for evaluation)
    forecast_history = model.predict(df[['ds', 'weekday', 'month', 'is_weekend']])
    
    # 4. Combine and Inverse Log-Transform
    # Select only the relevant columns from history and future
    cols_to_keep = ['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend']
    forecast = pd.concat([
        forecast_history[cols_to_keep], 
        forecast_future[cols_to_keep]
    ], ignore_index=True)
    
    # Inverse log-transform for all columns
    for col in ['yhat', 'yhat_lower', 'yhat_upper']:
        forecast[col] = np.expm1(forecast[col])

    print(f"Generated {len(forecast)} total days of forecasts (history + {periods} future)")

    return forecast



In [0]:
# ==============================================================================
# 6. EVALUATE MODEL AND LOG METRICS
# ==============================================================================

def evaluate_and_log_metrics(df_original: pd.DataFrame, forecast: pd.DataFrame, target_metric: str) -> float:
    """
    Evaluate Prophet forecast vs historical actuals and log metrics.
    Handles Databricks/Spark nested types, list-wrapped values, dicts, and dtype issues.
    """
    # ------------------------------------------------------------
    # 1. Select matching historical data (exclude future forecast)
    # ------------------------------------------------------------
    cutoff_date = forecast['ds'].max() - pd.Timedelta(days=FORECAST_DAYS)
    df_history = df_original[df_original['ds'] <= cutoff_date].copy()

    # ------------------------------------------------------------
    # 2. Merge historical with forecasted yhat
    # ------------------------------------------------------------
    comparison = df_history.merge(
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']],
        on='ds', 
        how='inner'
    )

    # Use the raw non-log-transformed target
    if 'y_raw' in comparison.columns:
        comparison['y'] = comparison['y_raw']
        comparison.drop(columns=['y_raw'], inplace=True)

    # ------------------------------------------------------------
    # 3. Flatten function for nested lists/arrays/dicts
    # ------------------------------------------------------------
    def flatten_to_scalar(x):
        if isinstance(x, (list, tuple, np.ndarray)):
            if len(x) == 0:
                return np.nan
            return flatten_to_scalar(x[0])
        if isinstance(x, dict):
            if len(x) == 0:
                return np.nan
            return flatten_to_scalar(list(x.values())[0])
        if hasattr(x, "asDict"):  # Spark Row
            vals = list(x.asDict().values())
            return flatten_to_scalar(vals[0]) if vals else np.nan
        # If it's already a scalar, return it
        return x

    # ------------------------------------------------------------
    # 4. Convert to numeric safely - ELEMENT BY ELEMENT
    # ------------------------------------------------------------
    # First flatten, then convert
    comparison['y'] = comparison['y'].apply(flatten_to_scalar)
    comparison['yhat'] = comparison['yhat'].apply(flatten_to_scalar)
    
    # Convert using a safer approach that maintains index alignment
    def safe_convert_to_float(val):
        try:
            return float(val)
        except (TypeError, ValueError):
            return np.nan
    
    comparison['y'] = comparison['y'].apply(safe_convert_to_float)
    comparison['yhat'] = comparison['yhat'].apply(safe_convert_to_float)

    # Drop rows where conversion failed
    initial_len = len(comparison)
    comparison.dropna(subset=['y', 'yhat'], inplace=True)
    final_len = len(comparison)
    
    if initial_len != final_len:
        print(f"Dropped {initial_len - final_len} rows due to NaN values")
    
    # Ensure we have data to evaluate
    if len(comparison) == 0:
        raise ValueError("No valid data points after cleaning. Check data types in your DataFrame.")

    # ------------------------------------------------------------
    # 5. Apply floor to avoid divide-by-zero in MAPE/SMAPE
    # ------------------------------------------------------------
    comparison['y'] = comparison['y'].clip(lower=100)
    comparison['yhat'] = comparison['yhat'].clip(lower=100)

    # ------------------------------------------------------------
    # 6. Compute error metrics
    # ------------------------------------------------------------
    comparison['error'] = comparison['y'] - comparison['yhat']
    comparison['abs_error'] = comparison['error'].abs()

    mae  = comparison['abs_error'].mean()
    mape = (comparison['abs_error'] / comparison['y']).mean() * 100
    rmse = np.sqrt((comparison['error'] ** 2).mean())
    smape = (
        100 / len(comparison) *
        np.sum(
            2 * np.abs(comparison['y'] - comparison['yhat']) /
            (np.abs(comparison['y']) + np.abs(comparison['yhat']))
        )
    )

    # ------------------------------------------------------------
    # 7. Log metrics to MLflow
    # ------------------------------------------------------------
    metrics = {'MAE': mae, 'MAPE': mape, 'RMSE': rmse, 'SMAPE': smape}
    mlflow.log_metrics({f"eval_{k}": float(v) for k, v in metrics.items()})

    # ------------------------------------------------------------
    # 8. Pretty print metrics
    # ------------------------------------------------------------
    print("="*60)
    print("MODEL PERFORMANCE METRICS")
    print("="*60)
    print(f"MAE  (Mean Absolute Error):          ${mae:,.2f}")
    print(f"MAPE (Mean Absolute % Error):        {mape:.2f}%")
    print(f"RMSE (Root Mean Squared Error):      ${rmse:,.2f}")
    print(f"SMAPE (Symmetric Mean Abs % Error):  {smape:.2f}%")
    print("="*60)

    return metrics[target_metric.upper()]

In [0]:
# ==============================================================================
# 7. EXECUTION BLOCK (MLFLOW INTEGRATION)
# ==============================================================================

def main_prophet_run(input_table, forecast_type, forecast_days, target_metric):
    """
    Main function to run the Prophet model pipeline and log results to MLflow.
    """
    # Use a specific run name
    run_name = f"Prophet_Forecast_{forecast_type}"
    with mlflow.start_run(run_name=run_name) as run:
        
        print(f"--- Starting Prophet Pipeline for {forecast_type} ---")
        
        # 1. Load and Prepare Data
        df = read_and_prepare_data(input_table, forecast_type, spark)
        
        # 2. Log Model Parameters to MLflow
        mlflow.log_params({
            "forecast_type": forecast_type,
            "forecast_days": forecast_days,
            "seasonality_mode": 'multiplicative',
            "changepoint_prior_scale": 0.05,
            "seasonality_prior_scale": 10,
            "data_start_date": df['ds'].min().strftime('%Y-%m-%d'),
            "data_end_date": df['ds'].max().strftime('%Y-%m-%d')
        })
        
        # 3. Train Model
        model = train_prophet_model(df)
        
        # 4. Generate Forecasts (including historical predictions for evaluation)
        forecast = generate_and_postprocess_forecast(model, df, periods=forecast_days)
        
        # 5. Evaluate and Log Metrics
        metric_value = evaluate_and_log_metrics(df, forecast, target_metric)
        
        # 6. Save Artifacts
        
        # a. Save Prophet Model
        model_path = "prophet_model.pkl"
        with open(model_path, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_path, "prophet_artifacts")
        
        # b. Save Forecast CSV (for reporting)
        output_df = forecast.tail(forecast_days).copy()
        output_df = output_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend']]
        output_df.columns = ['date', 'predicted_value', 'lower_bound', 'upper_bound', 'trend']
        output_df['model_run_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        filename_csv = f'prophet_forecast_future.csv'
        output_df.to_csv(filename_csv, index=False)
        mlflow.log_artifact(filename_csv, "forecast_output")
        
        print(f"\nModel and results logged to MLflow run: {run.info.run_uuid}")

        # 7. Visualization
        fig = model.plot(forecast, figsize=(12, 6))
        plt.title(f'Prophet Forecast for {forecast_type.replace("_", " ").title()}', fontsize=14)
        
        # Log the figure to MLflow
        fig_path = f"forecast_plot_{forecast_type}.png"
        fig.savefig(fig_path)
        mlflow.log_artifact(fig_path, "visualizations")
        plt.close(fig) # Closing plot to prevent memory leaks

        print(f"\nProphet Pipeline Completed Successfully!")
        
        return forecast

In [0]:
# ==============================================================================
# 7. EXECUTION OF THE MAIN RUN
# ==============================================================================

from prophet import Prophet 

final_forecast = main_prophet_run(INPUT_TABLE_NAME, FORECAST_TYPE, FORECAST_DAYS, TARGET_METRIC)

# Snippet of the final forecast
display(final_forecast.tail(FORECAST_DAYS))


--- Starting Prophet Pipeline for daily_revenue ---
Reading data from PostgreSQL table: retail_feature_engg_done...


21:23:02 - cmdstanpy - INFO - Chain [1] start processing


Loaded 104023 records...
✓ Filtered to start from: 2017-09-01 00:00:00
Aggregated to daily revenue: 364 days
✓ Target variable range: 7.55 to 11.10
Added Brazilian Holidays
Training Prophet Model


21:23:02 - cmdstanpy - INFO - Chain [1] done processing


Model trained successfully
Generated 394 total days of forecasts (history + 30 future)
MODEL PERFORMANCE METRICS
MAE  (Mean Absolute Error):          $5,065.48
MAPE (Mean Absolute % Error):        16.54%
RMSE (Root Mean Squared Error):      $6,697.59
SMAPE (Symmetric Mean Abs % Error):  15.78%

Model and results logged to MLflow run: bac20731bceb42229f9b064ac47557fd

✨ Prophet Pipeline Completed Successfully!
🏃 View run Prophet_Forecast_daily_revenue at: https://dbc-9a4f847e-bb18.cloud.databricks.com/ml/experiments/2912528436743962/runs/bac20731bceb42229f9b064ac47557fd
🧪 View experiment at: https://dbc-9a4f847e-bb18.cloud.databricks.com/ml/experiments/2912528436743962


ds,yhat,yhat_lower,yhat_upper,trend
2018-09-04T00:00:00Z,3080.4098815001207,2388.6503141395406,4017.48509934458,9.015676393425144
2018-09-05T00:00:00Z,2653.170236476089,2008.6230760768965,3510.449588074229,9.0078954448152
2018-09-06T00:00:00Z,2492.3876872312167,1932.186242268804,3249.139347769086,9.000114496205256
2018-09-07T00:00:00Z,1656.537559803055,1274.588201262273,2201.392827919716,8.992333547595313
2018-09-08T00:00:00Z,1675.4351142173016,1264.0777627523203,2152.5336815388187,8.984552598985369
2018-09-09T00:00:00Z,1714.8331520016432,1305.8546457892646,2267.844571471202,8.976771650375424
2018-09-10T00:00:00Z,2158.085605427028,1641.7752118005774,2843.0570950553156,8.968990701765481
2018-09-11T00:00:00Z,2085.664905481449,1588.1965992566672,2706.8208552519523,8.961209753155536
2018-09-12T00:00:00Z,1921.6158319030737,1468.7012239815888,2521.5030116484204,8.953428804545593
2018-09-13T00:00:00Z,1925.5508100555749,1495.179780814303,2530.926943103864,8.94564785593565
